<a href="https://colab.research.google.com/github/kairamilanifitria/NLP-Projects/blob/main/Project%202%20Text%20Summarization/Project2_Training_Failed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
train_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/reformed_dataset/train_reformed.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/reformed_dataset/test_reformed.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/reformed_dataset/dev_reformed.csv')

# Basic overview
print(train_df.shape)
print(train_df.info())
print(train_df.isnull().sum())

(193883, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193883 entries, 0 to 193882
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         193883 non-null  int64 
 1   url        193883 non-null  object
 2   text       193883 non-null  object
 3   summary    193883 non-null  object
 4   e_summary  193883 non-null  object
dtypes: int64(1), object(4)
memory usage: 7.4+ MB
None
id           0
url          0
text         0
summary      0
e_summary    0
dtype: int64


In [2]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00


In [7]:
!pip install transformers

# Tokenizing

In [22]:
# Import necessary libraries
import pandas as pd
import torch
from transformers import BertTokenizer, EncoderDecoderModel, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq
from transformers import pipeline

In [33]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

def preprocess_data(examples, max_length=512, stride=256):
    # Handling cases of samples longer than max_length
    model_inputs = tokenizer(examples['text'],
                            max_length=max_length,
                            truncation=True,
                            stride=stride,
                            padding="max_length",
                            # Disable returning overflowing tokens
                            return_overflowing_tokens=False,
                            return_tensors="pt")

    # Since return_overflowing_tokens is disabled,
    # each example maps to itself
    sample_mapping = list(range(len(examples["text"])))

    # Process labels with the same mapping
    model_inputs["labels"] = [
        tokenizer(
            examples["summary"][i],
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        ).input_ids
        for i in sample_mapping
    ]

    # Flatten the labels
    model_inputs["labels"] = [item for sublist in model_inputs["labels"] for item in sublist]

    return model_inputs

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
# Preprocess the datasets
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(preprocess_data, batched=True)

dev_dataset = Dataset.from_pandas(dev_df)
dev_dataset = dev_dataset.map(preprocess_data, batched=True)

test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/193883 [00:00<?, ? examples/s]

Map:   0%|          | 0/10972 [00:00<?, ? examples/s]

Map:   0%|          | 0/10972 [00:00<?, ? examples/s]

In [35]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [36]:
# Abstractive Summarization using Encoder-Decoder Model (BART or T5)
abstractive_model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization").to(device)  # Load model to GPU if available

In [16]:
# Install transformers if necessary
!pip install transformers datasets accelerate

In [37]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, BertTokenizer, EncoderDecoderModel
import torch
from datasets import Dataset

In [39]:
# Training arguments for abstractive summarization
training_args_abs = TrainingArguments(
    output_dir='./results_abs',
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=200,
    learning_rate=5e-5,
    # predict_with_generate=True, # Remove this line as it's not a valid argument for TrainingArguments
    fp16=True if device == 'cuda' else False
)

# Instead of passing 'predict_with_generate' to TrainingArguments,
# set it as an argument when calling the 'predict' method of the Trainer:
# predictions = trainer_abs.predict(test_dataset, metric_key_prefix="predict",  max_length=128, num_beams=4) # Example
# More details: https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.predict

# Data collator for seq2seq tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=abstractive_model)

# Trainer for abstractive summarization
trainer_abs = Trainer(
    model=abstractive_model,
    args=training_args_abs,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [40]:
# Train and evaluate the abstractive model
trainer_abs.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss
500,0.174200,0.189364
1000,0.169400,0.179652
1500,0.168200,0.179179
2000,0.167100,0.184733
2500,0.161900,0.174319
3000,0.165700,0.174316
3500,0.158000,0.177128
4000,0.161100,0.172611
4500,0.164500,0.172529
5000,0.152700,0.173444


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Some

KeyboardInterrupt: 

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

NameError: name 'model' is not defined